In [0]:
service_credential = dbutils.secrets.get(scope="Tokyo-Olympic",key="TokyoSecret")

spark.conf.set("fs.azure.account.auth.type.tokyoolympicchayuthpong.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.tokyoolympicchayuthpong.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.tokyoolympicchayuthpong.dfs.core.windows.net", "7610b0d7-eba2-449c-8336-e72bbcd4d19e")
spark.conf.set("fs.azure.account.oauth2.client.secret.tokyoolympicchayuthpong.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.tokyoolympicchayuthpong.dfs.core.windows.net", "https://login.microsoftonline.com/5867d0d2-9189-4c1c-8788-6a37e00f88f5/oauth2/token")

In [0]:
entriesgender_df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/raw-data/EntriesGender.csv",header=True, inferSchema=True)
athletes_df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/raw-data/athletes.csv", header=True, inferSchema=True)
coaches_df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/raw-data/coaches.csv", header=True, inferSchema=True)
medals_df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/raw-data/medals.csv", header=True, inferSchema=True)
teams_df = spark.read.csv("abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/raw-data/teams.csv", header=True, inferSchema=True)

In [0]:
display(entriesgender_df)

Discipline,Female,Male,Total
3x3 Basketball,32,32,64
Archery,64,64,128
Artistic Gymnastics,98,98,196
Artistic Swimming,105,0,105
Athletics,969,1072,2041
Badminton,86,87,173
Baseball/Softball,90,144,234
Basketball,144,144,288
Beach Volleyball,48,48,96
Boxing,102,187,289


In [0]:
# Find the top countries with highest number of gold medals
top_gold_medal_countries = medals_df.orderBy("Gold", ascending=False).select("Team/NOC","Gold").show()

+--------------------+----+
|            Team/NOC|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows


In [0]:
# Calculate the average number of entries by gender for each discipline
average_entries_by_gender = entriesgender_df.withColumn(
    'Avg_Female', entriesgender_df['Female'] / entriesgender_df['Total']
).withColumn(
    'Avg_Male', entriesgender_df['Male'] / entriesgender_df['Total']
)

average_entries_by_gender.show()

+--------------------+------+----+-----+-------------------+-------------------+
|          Discipline|Female|Male|Total|         Avg_Female|           Avg_Male|
+--------------------+------+----+-----+-------------------+-------------------+
|      3x3 Basketball|    32|  32|   64|                0.5|                0.5|
|             Archery|    64|  64|  128|                0.5|                0.5|
| Artistic Gymnastics|    98|  98|  196|                0.5|                0.5|
|   Artistic Swimming|   105|   0|  105|                1.0|                0.0|
|           Athletics|   969|1072| 2041| 0.4747672709456149| 0.5252327290543851|
|           Badminton|    86|  87|  173|0.49710982658959535| 0.5028901734104047|
|   Baseball/Softball|    90| 144|  234|0.38461538461538464| 0.6153846153846154|
|          Basketball|   144| 144|  288|                0.5|                0.5|
|    Beach Volleyball|    48|  48|   96|                0.5|                0.5|
|              Boxing|   102

In [0]:
athletes_df.repartition(1).write.mode("overwrite").option("header", "true").csv(
    "abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/transformed-data/athletes"
)

entriesgender_df.repartition(1).write.mode("overwrite").option("header", "true").csv(
    "abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/transformed-data/entriesgender"
)

coaches_df.repartition(1).write.mode("overwrite").option("header", "true").csv(
    "abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/transformed-data/coaches"
)

medals_df.repartition(1).write.mode("overwrite").option("header", "true").csv(
    "abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/transformed-data/medals"
)

teams_df.repartition(1).write.mode("overwrite").option("header", "true").csv(
    "abfss://tokyo-olympic-data@tokyoolympicchayuthpong.dfs.core.windows.net/transformed-data/teams"
)